In [ ]:
import numpy as np
import scipy.signal as signal
import torchaudio
from IPython.display import Audio
import torch
from matplotlib import pyplot as plt
import soundfile as sf
import time

In [ ]:
# THIS NOTEBOOK: 

# What is exactly the relation between the audio convolution used to, for example, add reverb to a signal,
# and the pytorch Conv1d used in a convolutional neural network?
# I wanted to understand this, because sometimes it might be needed to do audio convolution on gpu and 
# for that pytorch functions have to be used. For that a torchaudio function is good cause it can be used just as
# a typical numpy or scipy convolution. 
# It can also important to know this if we want to have a trainable impulse response in a convolutional neural network. 

In [ ]:
# Define the convolution functions
def convolve_scipy(sig, ir):
    return signal.fftconvolve(sig, ir,mode="full")

def convolve_numpy(sig, ir):
    return np.convolve(sig, ir,mode="full")

def convolve_torchaudio(sig, ir, device="cpu"):
    sig_tensor = torch.tensor(sig, dtype=torch.float32).to(device)
    ir_tensor = torch.tensor(ir, dtype=torch.float32).to(device)
    return torchaudio.functional.convolve(sig_tensor, ir_tensor)

def convolve_torch1(sig, ir,device="cpu"):
    # sig - audio signal to be convolved (in conv1d terms - input tensor)
    # ir - impulse response to colvolve the signal with (in conv1d terms - convolution kernel)

    # torch.nn.functional.conv1d performs autocorrelation operation. To make it the actual convolution, the 
    # kernel has to be flipped. We have to make a copy of the flipped variable, otherwise we will get the 
    # error saying that the stride in numpy array is negative, and tensors with negative strides are not supported: 
    ir_flipped=ir[::-1].copy()
    # prepare the shape of the sig and ir so that it matches what conv1d expects:
    sig_tensor = torch.tensor(sig, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: (batch_size, in_channels, input_length)
    ir_tensor = torch.tensor(ir_flipped, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: (batch_size, in_channels, input_length)
    # create padding that allows for obtaining a "full" convolution, where the size of the 
    # convolved output signal is N+M-1 , where N=len(sig), M=len(ir)
    full_padding = len(ir_tensor[0][0]) - 1
    sig_tensor = torch.nn.functional.pad(sig_tensor, (full_padding, full_padding), mode='constant', value=0)
    return torch.nn.functional.conv1d(sig_tensor, ir_tensor)

def convolve_torch2(sig, ir, device="cpu"):
    # sig - audio signal to be convolved (in conv1d terms - input tensor)
    # ir - impulse response to colvolve the signal with (in conv1d terms - convolution kernel)

    # torch.nn.Conv1d performs autocorrelation operation. To make it the actual convolution, the 
    # kernel has to be flipped. We have to make a copy of the flipped variable, otherwise we will get the 
    # error saying that the stride in numpy array is negative, and tensors with negative strides are not supported. 
    ir_flipped=ir[::-1].copy()
    # prepare the shape of the sig and ir so that it matches what Conv1d expects:
    sig_tensor = torch.tensor(sig, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Shape: (batch_size, in_channels, input_length)
    ir_tensor = torch.tensor(ir_flipped.copy(), dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device) # Shape: (batch_size, in_channels, input_length)
    # create padding that allows for obtaining a "full" convolution, where the size of the 
    # convolved output signal is N+M-1 , where N=len(sig), M=len(ir)
    full_padding = len(ir_tensor[0][0]) - 1
    # define convolutional layer:
    conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=ir_tensor.shape[2], stride=1, padding=full_padding, bias=False)
    # Set the weights of the convolutional layer (kernel) to be the flipped impulse response:
    with torch.no_grad():
        conv_layer.weight.data = ir_tensor
        
    return conv_layer(sig_tensor)

# Other convolution tests:
# https://laurentperrinet.github.io/sciblog/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html





In [ ]:
# Generate or load sample audio signals:

# # generate:
# np.random.seed(0)
# signal1 = np.random.rand(1000)
# signal2 = np.random.rand(500)

# load:
signal1, fs1 = sf.read('audios/speech_VCTK_4_sentences.wav')
signal1=signal1[1:2*fs1]
signal2, fs2 = sf.read('audios/rir_ACE_lecture_hall.wav')



In [ ]:
# Test the convolution functions
start=time.time()
result_scipy = convolve_scipy(signal1, signal2)
end_scipy = time.time()
result_numpy = convolve_numpy(signal1, signal2)
end_numpy= time.time()
result_torchaudio = convolve_torchaudio(signal1, signal2).detach().numpy()
end_torchaudio= time.time()
result_torch1 = convolve_torch1(signal1, signal2).detach().numpy().squeeze(0).squeeze(0)
end_torch1=time.time()
result_torch2 = convolve_torch2(signal1, signal2).detach().numpy().squeeze(0).squeeze(0)
end_torch2=time.time()

print("scipy convolution time: " +str(end_scipy-start))
print("numpy convolution time: " +str(end_numpy-end_scipy))
print("torchaudio convolution time: " +str(end_torchaudio-end_numpy))
print("F.conv1d convolution time: " +str(end_torch1- end_torchaudio))
print("nn.Conv1d convolution time: " +str(end_torch2- end_torch1))


In [ ]:
print(f"{result_scipy.shape=}")
print(f"{result_numpy.shape=}")
print(f"{result_torchaudio.shape=}")
print(f"{result_torch1.shape=}")
print(f"{result_torch2.shape=}")

In [ ]:
print(np.allclose(result_scipy, result_torchaudio))
print(np.allclose(result_numpy, result_torchaudio))
print(np.allclose(result_numpy, result_torch2))
print(np.allclose(result_numpy, result_scipy))
print(np.allclose(result_numpy, result_torch1))
print(np.allclose(result_torch1, result_scipy))

In [ ]:
plt.figure()
plt.subplot(5,1,1);plt.plot(result_numpy);plt.title("numpy")
plt.subplot(5,1,2);plt.plot(result_scipy);plt.title("scipy")
plt.subplot(5,1,3);plt.plot(result_torchaudio);plt.title("torchaudio")
plt.subplot(5,1,4);plt.plot(result_torch1);plt.title("functional.conv1d")
plt.subplot(5,1,5);plt.plot(result_torch2);plt.title("nn.Conv1d")
plt.tight_layout()
plt.show()


In [ ]:
Audio(result_torch2,rate=48000)